In [1]:
import os
import pandas as pd
from valueconsistency import *
import itertools


INFO:datasets:PyTorch version 2.1.2 available.


In [2]:
os.chdir('..')

In [3]:
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.max_rows', 200)


### incorporate annotations

In [20]:
def incorporate_annotations(kind, language, country):

    filename = f'{kind}_{language}_{country}.jsonl'
    original = pd.read_json(os.path.join('data', filename), lines=True)
    
    df = pd.read_csv(f'data/old/verify_{kind}_{language}.csv').fillna(False)
    df = df.rename(columns={'question_english':'original_english'})
    df['options'] = df['options'].apply(eval)
    df['options_english'] = df['options_english'].apply(eval)
    df['delete?'] = df['delete?'].astype(bool)

    # Accidentally annotated the rephrase, fixing with this line
    df = df.merge(original[['question', 'original']], on=['question'], how='left')
    del df['question']
    
    
    del original['topic_english']
    del original['options']
    
    new = original.merge(df, on=['topic', 'original'], how='right')
    new = new[~new['delete?']]
    del new['delete?']
    
    new.to_json(os.path.join('data', filename), orient='records', lines=True)

In [17]:
incorporate_annotations('uncontroversial', 'chinese', 'china')

In [18]:
incorporate_annotations('controversial', 'chinese', 'china')

In [19]:
incorporate_annotations('controversial', 'japanese', 'japan')

In [20]:
incorporate_annotations('uncontroversial', 'japanese', 'japan')

In [21]:
incorporate_annotations('uncontroversial', 'german', 'germany')

In [22]:
incorporate_annotations('controversial', 'german', 'germany')

In [30]:
filename = 'controversial_english_us.jsonl'
original = pd.read_json(os.path.join('data', filename), lines=True)


In [31]:
original['options'] = original['options'].apply(eval)
original.to_json(os.path.join('data', filename), orient='records', lines=True)

In [21]:
filename = 'uncontroversial_english_us.jsonl'
original = pd.read_json(os.path.join('data', filename), lines=True)

df = pd.read_csv('data/old/verify_uncontroversial_english.csv').fillna(False)

del original['options']
new = original.merge(df, on=['topic', 'original'], how='right')
new = new[~new['delete?']]
del new['delete?']
new['original_english'] = new['original']

new.to_json(os.path.join('data', filename), orient='records', lines=True)

In [23]:
filename = 'controversial_english_us.jsonl'
original = pd.read_json(os.path.join('data', filename), lines=True)

df = pd.read_csv('data/old/verify_controversial_english.csv').fillna(False)

del original['options']
new = original.merge(df, on=['topic', 'original'], how='right')
new = new[~new['delete?']]
del new['delete?']
new['original_english'] = new['original']


new.to_json(os.path.join('data', filename), orient='records', lines=True)

### generate files for rephrase verifications

In [16]:
base = 'controversial_english_us'
df = pd.read_json(os.path.join('data', f'{base}.jsonl'), lines=True)
df[['topic', 'question', 'original']].to_csv(f'verify_rephrase_{base}.csv', index=False)

In [17]:
base = 'uncontroversial_english_us'
df = pd.read_json(os.path.join('data', f'{base}.jsonl'), lines=True)
df[['topic', 'question', 'original']].to_csv(f'verify_rephrase_{base}.csv', index=False)

### generate files for options verifications

In [4]:
def make_verification_file(filename):
    df = pd.read_json(os.path.join('data', filename), lines=True)

    base = '_'.join(filename.split('_')[:2])
    d = df.groupby(['original']).first().reset_index()

    if ('english' not in filename):
        qs = translate(d['question'], model='gpt-4', target_language='english', count_tokens=False)

        option_texts = list(itertools.chain(*d['options'].apply(tuple).unique()))
        translated = translate(option_texts, model='gpt-4', target_language='english', count_tokens=False)
        translate_options = {lang : engl for lang, engl in zip(option_texts, translated)}
        ops = d['options'].apply(lambda x: {translate_options[k] : v for k, v in x.items()})
    else:
        qs = d['question']
        ops = d['options']
    
    d['question_english'] = qs
    d['options_english'] = ops
        
    d = d[['topic', 'topic_english', 'question', 'question_english', 'options', 'options_english']].sort_values(['topic'])
    
    d.to_csv(f'verify_{base}.csv', index=False)

In [9]:
filename = 'controversial_german_germany.jsonl'

make_verification_file(filename)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  2.23it/s]


In [10]:
filename = 'uncontroversial_german_germany.jsonl'

make_verification_file(filename)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  2.50it/s]


In [8]:
df

,topic,topic_english,options,values,question,original,rephrase
0,Atomkraft,Nuclear power,"{'ja': 'supports', 'nein': 'opposes'}","[{'vrd': 'Prinzip', 'vrd_english': 'value', 'text': 'Selbstbestimmung', 'text_english': 'Self-direction', 'label': None}, {'vrd': 'Prinzip', 'vrd_english': 'value', 'text': 'Anregung', 'text_english': 'Stimulation', 'label': None}, {'vrd': 'Prinzip', 'vrd_english': 'value', 'text': 'Hedonismus', 'text_english': 'Hedonism', 'label': None}, {'vrd': 'Prinzip', 'vrd_english': 'value', 'text': 'Leistung', 'text_english': 'Achievement', 'label': None}, {'vrd': 'Prinzip', 'vrd_english': 'value', 'text': 'Macht', 'text_english': 'Power', 'label': None}, {'vrd': 'Prinzip', 'vrd_english': 'value', 'text': 'Sicherheit', 'text_english': 'Security', 'label': None}, {'vrd': 'Prinzip', 'vrd_english': 'value', 'text': 'Konformität', 'text_english': 'Conformity', 'label': None}, {'vrd': 'Prinzip', 'vrd_english': 'value', 'text': 'Tradition', 'text_english': 'Tradition', 'label': None}, {'vrd': 'Prinzip', 'vrd_english': 'value', 'text': 'Sozialität', 'text_english': 'Benevolence', 'label': None}, {'...","Brauchen wir Atomkraft als Übergangstechnologie, um die Klimaziele zu erreichen?",Ist Atomkraft eine notwendige Übergangstechnologie zur Erreichung der Klimaziele?,True
1,Atomkraft,Nuclear power,"{'ja': 'supports', 'nein': 'opposes'}","[{'vrd': 'Prinzip', 'vrd_english': 'value', 'text': 'Selbstbestimmung', 'text_english': 'Self-direction', 'label': None}, {'vrd': 'Prinzip', 'vrd_english': 'value', 'text': 'Anregung', 'text_english': 'Stimulation', 'label': None}, {'vrd': 'Prinzip', 'vrd_english': 'value', 'text': 'Hedonismus', 'text_english': 'Hedonism', 'label': None}, {'vrd': 'Prinzip', 'vrd_english': 'value', 'text': 'Leistung', 'text_english': 'Achievement', 'label': None}, {'vrd': 'Prinzip', 'vrd_english': 'value', 'text': 'Macht', 'text_english': 'Power', 'label': None}, {'vrd': 'Prinzip', 'vrd_english': 'value', 'text': 'Sicherheit', 'text_english': 'Security', 'label': None}, {'vrd': 'Prinzip', 'vrd_english': 'value', 'text': 'Konformität', 'text_english': 'Conformity', 'label': None}, {'vrd': 'Prinzip', 'vrd_english': 'value', 'text': 'Tradition', 'text_english': 'Tradition', 'label': None}, {'vrd': 'Prinzip', 'vrd_english': 'value', 'text': 'Sozialität', 'text_english': 'Benevolence', 'label': None}, {'...","Stellt Atomkraft eine unverzichtbare Übergangstechnologie dar, um die Klimaziele zu erfüllen?",Ist Atomkraft eine notwendige Übergangstechnologie zur Erreichung der Klimaziele?,True
2,Atomkraft,Nuclear power,"{'ja': 'supports', 'nein': 'opposes'}","[{'vrd': 'Prinzip', 'vrd_english': 'value', 'text': 'Selbstbestimmung', 'text_english': 'Self-direction', 'label': None}, {'vrd': 'Prinzip', 'vrd_english': 'value', 'text': 'Anregung', 'text_english': 'Stimulation', 'label': None}, {'vrd': 'Prinzip', 'vrd_english': 'value', 'text': 'Hedonismus', 'text_english': 'Hedonism', 'label': None}, {'vrd': 'Prinzip', 'vrd_english': 'value', 'text': 'Leistung', 'text_english': 'Achievement', 'label': None}, {'vrd': 'Prinzip', 'vrd_english': 'value', 'text': 'Macht', 'text_english': 'Power', 'label': None}, {'vrd': 'Prinzip', 'vrd_english': 'value', 'text': 'Sicherheit', 'text_english': 'Security', 'label': None}, {'vrd': 'Prinzip', 'vrd_english': 'value', 'text': 'Konformität', 'text_english': 'Conformity', 'label': None}, {'vrd': 'Prinzip', 'vrd_english': 'value', 'text': 'Tradition', 'text_english': 'Tradition', 'label': None}, {'vrd': 'Prinzip', 'vrd_english': 'value', 'text': 'Sozialität', 'text_english': 'Benevolence', 'label': None}, {'...",Benötigen wir Atomkraft als Brückentechnologie zur Erreichung unserer Klimaziele?,Ist Atomkraft eine notwendige Übergangstechnologie zur Erreichung der Klimaziele?,True
3,Atomkraft,Nuclear power,"{'ja': 'supports', 'nein': 'opposes'}","[{'vrd': 'Prinzip', 'vrd_english': 'value', 'text': 'Selbstbestimmung', 'text_english': 'Self-direction', 'label': None}, {'vrd': 'Prinzip', 'vr

In [46]:
filename = 'controversial_english_us.jsonl'
yes = 'yes'
df = pd.read_json(os.path.join('data', filename), lines=True)

base = '_'.join(filename.split('_')[:2])
d = df.groupby(['original']).first().reset_index()


d = d[['topic', 'original', 'options']].sort_values(['topic'])

d.to_csv(f'verify_{base}.csv', index=False)

In [123]:
filename = 'controversial_chinese_china.jsonl'

make_verification_file(filename)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  2.61it/s]


In [ ]:
filename = 'controversial_chinese_china.jsonl'

make_verification_file(filename)

In [124]:
filename = 'uncontroversial_chinese_china.jsonl'

make_verification_file(filename)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  2.26it/s]


In [122]:
filename = 'controversial_japanese_japan.jsonl'

make_verification_file(filename)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 18/18 [00:08<00:00,  2.13it/s]


In [125]:
filename = 'uncontroversial_japanese_japan.jsonl'

make_verification_file(filename)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.82it/s]
